## 作業

請參考昨天的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。請參考 Keras [官方文件中 fit_generator 的說明](https://keras.io/models/sequential/)

In [1]:
# Loading library
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# Setting hyper-parameters
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

In [3]:
# Check data formation
(x_train_orginal, y_train_orginal), (x_test_orginal, y_test_orginal) = cifar10.load_data()
print('x_train shape:', x_train_orginal.shape)
print('x_test shape:', x_test_orginal.shape)

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)


In [4]:
x_train = x_train_orginal
x_test = x_test_orginal

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train_orginal, num_classes)
y_test = to_categorical(y_test_orginal, num_classes)

In [9]:
import math

class Model():
    def __init__(self, x_train, y_train, x_test, y_test):
        self.FirConvSize = None
        self.SecConvSize = None
        self.Padding = None
        self.model = None
        self.KernlSize = None
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.cnt = 0
    
    def setHypParam(self, padding='same', KernlSize=(3, 3), FirConvSize=32, SecConvSize=64):
        self.FirConvSize = FirConvSize
        self.SecConvSize = SecConvSize
        self.KernlSize = KernlSize
        self.Padding = padding
    
    def buildModel(self):
        self.model = Sequential()
        self.model.add(Conv2D(self.FirConvSize, self.KernlSize, padding=self.Padding,
                         input_shape=x_train.shape[1:]))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(self.FirConvSize, self.KernlSize))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        
        self.model.add(Conv2D(self.SecConvSize, self.KernlSize, padding=self.Padding))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(self.SecConvSize, self.KernlSize))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))

        self.model.add(Flatten())
        self.model.add(Dense(512))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(num_classes))
        self.model.add(Activation('softmax'))
    
    def setCompile(self):
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=RMSprop(),
                           metrics=['accuracy'])
        
    def train_generator(self, batch_num = 32):
        while True:
            for indexs in range(0, len(self.x_train), batch_num):
                x_train_gen = self.x_train[indexs: indexs+batch_num]
                y_train_gen = self.y_train[indexs: indexs+batch_num]
                yield x_train_gen, y_train_gen
  
    def fitModel_generator(self):
        self.cnt += 1
        print(self.cnt)
        self.model.fit_generator(self.train_generator(),
                                 epochs=epochs,
                                 steps_per_epoch=math.ceil(len(x_train)/32),
                                 verbose=True,
                                 validation_data=(self.x_test, self.y_test))
    
    def evaluateScore(self):
        return (self.model.evaluate(self.x_test, self.y_test, verbose=0))

In [10]:
# Discissing effection of conv size.
# setting hyper-parameters
FirConvSize = 64
SecConvSize = 64
padding = 'same'
KernelSize_list = (3, 3)

model = Model(x_train, y_train, x_test, y_test)
model.setHypParam(FirConvSize=FirConvSize, SecConvSize=SecConvSize)
model.buildModel()
model.setCompile()
model.fitModel_generator()
score = model.evaluateScore()
print('Test loss:%.4f' %score[0])
print('Test accuracy:%.4f' %score[1])

1
Epoch 1/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.5183 - acc: 0.4586 - val_loss: 1.1890 - val_acc: 0.5820
Epoch 2/10
1563/1563 [==============================] - 28s 18ms/step - loss: 1.1181 - acc: 0.6123 - val_loss: 1.0308 - val_acc: 0.6458
Epoch 3/10
1563/1563 [==============================] - 30s 19ms/step - loss: 1.0049 - acc: 0.6522 - val_loss: 1.0362 - val_acc: 0.6423
Epoch 4/10
1563/1563 [==============================] - 27s 18ms/step - loss: 0.9688 - acc: 0.6716 - val_loss: 1.0934 - val_acc: 0.6386
Epoch 5/10
1563/1563 [==============================] - 28s 18ms/step - loss: 0.9767 - acc: 0.6757 - val_loss: 0.9278 - val_acc: 0.6968
Epoch 6/10
1563/1563 [==============================] - 28s 18ms/step - loss: 1.0183 - acc: 0.6675 - val_loss: 0.9769 - val_acc: 0.6845
Epoch 7/10
1563/1563 [==============================] - 27s 18ms/step - loss: 1.0478 - acc: 0.6622 - val_loss: 1.1319 - val_acc: 0.6176
Epoch 8/10
1563/1563 [========================

# Conclusion
* 相較於 HW_97，這次在每次訓練的時候只用了 32 張圖進行訓練，因此得到的成效不如使用大量 Data 進行訓練。